In [1]:
import os
import pandas as pd
import numpy as np
from scipy import stats
from pathlib import Path
from dotenv import load_dotenv

from src.metrics import pearson_metric

load_dotenv()

True

In [2]:
dataset_dir = Path(os.environ['dataset_dir'])
data = pd.read_csv(dataset_dir / 'train.csv')

In [3]:
data = data.set_index('row_id')

In [4]:
# train on earlier data, test on later data
train = data[data.time_id < 200]
val = data.query("200 <= time_id < 210")
test = data.query("210 <= time_id < 300")

x_train, y_train = train.drop('target', axis=1), train[['time_id', 'target']]
x_val, y_val = val.drop('target', axis=1), val[['time_id', 'target']]
x_test, y_test = test.drop('target', axis=1), test[['time_id', 'target']]

In [5]:
from catboost import CatBoostRegressor, Pool

train_pool = Pool(x_train.drop('time_id', axis=1), y_train.target, cat_features=['investment_id'])
val_pool = Pool(x_val.drop('time_id', axis=1), y_val.target, cat_features=['investment_id'])

model = CatBoostRegressor(
    iterations=100,
    random_state=0,
    cat_features=['investment_id'], 
    verbose=20
)
model = model.fit(train_pool, eval_set=val_pool)

Learning rate set to 0.5
0:	learn: 0.9306783	test: 0.9365735	best: 0.9365735 (0)	total: 711ms	remaining: 1m 10s
20:	learn: 0.9178891	test: 0.9406778	best: 0.9357778 (9)	total: 11.1s	remaining: 41.6s
40:	learn: 0.9092868	test: 0.9372681	best: 0.9357778 (9)	total: 21.5s	remaining: 31s
60:	learn: 0.9020813	test: 0.9394631	best: 0.9357778 (9)	total: 32.5s	remaining: 20.8s
80:	learn: 0.8958848	test: 0.9394860	best: 0.9357778 (9)	total: 42.9s	remaining: 10.1s
99:	learn: 0.8902457	test: 0.9388778	best: 0.9357778 (9)	total: 52.9s	remaining: 0us

bestTest = 0.9357777902
bestIteration = 9

Shrink model to first 10 iterations.


In [6]:
y_pred = model.predict(x_test)
y_pred = pd.DataFrame({'target': y_pred}, index=x_test.index)

In [7]:
pearson_metric(y_test, y_pred)

0.12388904140526928

I thought the model failed to learn anything on features only: it overfits after 9th epoch.  
But metric is greater than 0, hmm